Import libraries

In [ ]:
# !pip install fiftyone
# !pip install umap-learn
# !pip install ipywidgets>=7.5
# !pip install opencv-python-headless==4.5.4.60 fiftyone

In [3]:
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import os

%matplotlib inline
import utils
import wandb

from IPython.display import Image,Video, clear_output  # to display images
clear_output()

In [5]:
# Configure wandb for monitoring and logging
# wandb.init()

In [6]:
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-164-g177da7f torch 1.11.0 CPU


Setup complete ✅ (10 CPUs, 16.0 GB RAM, 229.5/926.4 GB disk)


In [7]:
#git clone repo for first time
# !git clone https://github.com/ultralytics/yolov5.git
#git pull and update repo if prompted
# !git pull https://github.com/ultralytics/yolov5.git
#change directory to where yolov5 is located
%cd /Users/ethankuch/my_project

/Users/ethankuch/my_project


Extract custom ingredient object class from recipes

In [8]:
import openimages

In [9]:
import pandas as pd

file = "/Users/ethankuch/my_project/ingredient.csv"
df =pd.read_csv(file)
ingridentList = df["nutrition_key"].unique().tolist()

print("Number of Ingredient in List: ",len(ingridentList))
print("List of Ingredient: ",ingridentList)

Number of Ingredient in List:  430
List of Ingredient:  ['butter', 'chicken', 'onions', 'carrot', 'celery', 'garlic', 'flour', 'barbecue sauce', 'potatoes', 'pepper', 'cream', 'blue cheese', 'mayonnaise', 'cheddar cheese', 'bread', 'salt', 'lemon juice', 'olive oil', 'avocado', 'honey', 'kale', 'berries', 'fennel', 'almonds', 'beef', 'mixed vegetables', 'broth', 'ground beef', 'oregano', 'basil', 'tomatoes', 'wine', 'zzseasoning', 'milk', 'parmesan cheese', 'roast beef', 'thyme', 'rosemary', 'cauliflower', 'parsley', 'half and half', 'sour cream', 'bacon', 'trail mix', 'cheese', 'apple cider', 'dumplings', 'cornmeal', 'protein powder', 'quinoa', 'water', 'zucchini', 'black beans', 'kidney beans', 'chili', 'noodles', 'ground chicken', 'oil', 'green beans', 'cabbage', 'edamame', 'cucumbers', 'soy sauce', 'vinegar', 'cilantro', 'seeds', 'ginger', 'roasted potatoes', 'lime', 'brown sugar', 'coconut milk', 'tofu', 'broccoli', 'corn on the cob', 'vegetable oil', 'corn', 'lemon', 'stews', 'su

In [10]:
import fiftyone as fo
import fiftyone.zoo as foz
from difflib import get_close_matches
import numpy as np

# List of approved labels in the fo dataset
# fo_list=fo.utils.openimages.getclasses()
fo_list=openimages.get_classes()
avaliable_fo =list()
i = 0 
while i < len(ingridentList): 

  matches=get_close_matches(ingridentList[i],fo_list,n=2,cutoff=0.75)
  
  avaliable_fo.extend(matches)  
  i += 1
  avaliable_fo.sort()
print("Number of ingredient classes avaliable on OpenImageV6: ",len(avaliable_fo))
print(avaliable_fo)

Number of ingredient classes avaliable on OpenImageV6:  65
['Apple', 'Artichoke', 'Banana', 'Beer', 'Bell pepper', 'Bread', 'Broccoli', 'Cabbage', 'Cantaloupe', 'Carrot', 'Cheese', 'Chicken', 'Coat', 'Coconut', 'Coffee', 'Cookie', 'Crab', 'Cream', 'Croissant', 'Cucumber', 'Deer', 'Dice', 'Drum', 'Duck', 'Fish', 'Fork', 'French fries', 'Goat', 'Grapefruit', 'Guacamole', 'Hamburger', 'Hamburger', 'Hot dog', 'Ice cream', 'Lemon', 'Lobster', 'Maple', 'Milk', 'Muffin', 'Mushroom', 'Orange', 'Oyster', 'Pancake', 'Parrot', 'Pasta', 'Peach', 'Pineapple', 'Pizza', 'Plate', 'Platter', 'Pomegranate', 'Popcorn', 'Poster', 'Poster', 'Pretzel', 'Pumpkin', 'Seafood', 'Shrimp', 'Squid', 'Sushi', 'Turkey', 'Watermelon', 'Wheel', 'Wine', 'Zucchini']


In [11]:
# Loading matched Ingredient from recipe and OpenImagev6 database to train customized object detection
dataset = foz.load_zoo_dataset(
    "open-images-v6", 
    split="train", 
    label_types=["detections", "classifications"], 
    classes = avaliable_fo,
    max_samples=3000,
    seed=69,
    shuffle=True,
    dataset_name="df_train",
)

Only found 987 (<3000) samples matching your requirements
Necessary images already downloaded
Existing download of split 'train' is sufficient
Loading 'open-images-v6' split 'train'
 100% |█████████████████| 987/987 [7.4s elapsed, 0s remaining, 125.1 samples/s]      
Dataset 'df_train' created


In [12]:
dataset_val = foz.load_zoo_dataset(
    "open-images-v6", 
    split="validation", 
    label_types=["detections", "classifications"], 
    classes = avaliable_fo,
    max_samples=900,
    seed=69,
    shuffle=True,
    dataset_name="df_val",
)

Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading 'open-images-v6' split 'validation'
 100% |█████████████████| 900/900 [7.3s elapsed, 0s remaining, 134.7 samples/s]      
Dataset 'df_val' created


In [13]:
export_dir = "/Users/ethankuch/my_project/yolov5"
label_field = "detections"  # for example

# The Dataset or DatasetView to export
dataset_or_view = dataset

# Export the dataset
dataset_or_view.export( 
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    classes=avaliable_fo,
    label_field=label_field,
    split="train"
)
clear_output()
#Undo above clear_output to see the list of Ignored detection labels used for training

In [14]:
export_dir = "/Users/ethankuch/my_project/yolov5"
label_field = "detections"  # for example

# The Dataset or DatasetView to export
dataset_or_view = dataset_val

# Export the dataset
dataset_or_view.export( 
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    classes=avaliable_fo,
    label_field=label_field,
    split="val"
)
clear_output()
#Undo above clear_output to see the list of Ignored detection labels used for validation

In [15]:
%cd yolov5

/Users/ethankuch/my_project/yolov5


In [16]:
# If prompted to update github, uncomment the below line to update
# !git pull https://github.com/ultralytics/yolov5

In [17]:
# !python train.py --img 640 --batch 50 --epochs 10 --data /Users/ethankuch/my_project/yolov5/models/dataset.yaml --weights /Users/ethankuch/my_project/yolov5/yolov5m6.pt
!python train.py --img 320 --batch 60 --epochs 100 --project 'foodscan' --save-period 1 --upload_dataset --data /Users/ethankuch/my_project/yolov5/models/dataset.yaml --weights /Users/ethankuch/my_project/yolov5/yolov5s.pt --cache

Retry attempt failed:
Traceback (most recent call last):
  File "/Users/ethankuch/opt/anaconda3/envs/visvenv/lib/python3.7/site-packages/urllib3/connection.py", line 175, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/Users/ethankuch/opt/anaconda3/envs/visvenv/lib/python3.7/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/ethankuch/opt/anaconda3/envs/visvenv/lib/python3.7/socket.py", line 752, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ethankuch/opt/anaconda3/envs/visvenv/lib/python3.7/site-packages/urllib3/connectionpool.py", line 710, in urlopen
    chunked=chunked,
  File "/Users/ethankuch/opt/anaconda3

In [18]:
wandb.login()

Retry attempt failed:
Traceback (most recent call last):
  File "/Users/ethankuch/opt/anaconda3/envs/visvenv/lib/python3.7/site-packages/urllib3/connection.py", line 175, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/Users/ethankuch/opt/anaconda3/envs/visvenv/lib/python3.7/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/ethankuch/opt/anaconda3/envs/visvenv/lib/python3.7/socket.py", line 752, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ethankuch/opt/anaconda3/envs/visvenv/lib/python3.7/site-packages/urllib3/connectionpool.py", line 710, in urlopen
    chunked=chunked,
  File "/Users/ethankuch/opt/anaconda3

wandb: Network error (ConnectionError), entering retry loop.
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


True

Detection of Images/Video

In [24]:
!python detect.py --img 320 --data /Users/ethankuch/my_project/yolov5/models/dataset.yaml --weights yolov5x.pt  --conf 0.4 --source data/foodimage

detect: weights=['yolov5x.pt'], source=data/foodimage, data=/Users/ethankuch/my_project/yolov5/models/dataset.yaml, imgsz=[320, 320], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-164-g177da7f torch 1.11.0 CPU

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
image 1/6 /Users/ethankuch/my_project/yolov5/data/foodimage/fs1.jpeg: 224x320 8 bottles, 1 banana, 4 apples, 3 oranges, 2 broccolis, 1 refrigerator, Done. (1.253s)
image 2/6 /Users/ethankuch/my_project/yolov5/data/foodimage/fs2.jpeg: 320x224 1 banana, 4 apples, 10 oranges, 3 broccolis, 2 carrots, Done. (1.244s)
image 3/6 /Users/ethankuch/my_project/yolov5/data/foodimage/fs3.jpeg: 320x224 2 bottles, 1

In [20]:
exp=max(glob.glob(os.path.join('/Users/ethankuch/my_project/yolov5/runs/detect/', '*/')), key=os.path.getmtime)
exp = os.path.normpath(exp)

In [21]:
#Update export count
detect_path = exp
images = []
for img_path in glob.glob(detect_path + '/*.jpeg'):
    images.append(mpimg.imread(img_path))

plt.figure(figsize=(40,40))
columns = 4
for i, image in enumerate(images):
    plt.subplot((int(len(images) / columns + 1)), columns, i + 1)
    plt.tick_params(left = False, right = False , labelleft = False, labelbottom = False, bottom = False)
    plt.imshow(image)


<Figure size 2880x2880 with 0 Axes>

In [ ]:
# !python prepare_csv.py --label_path /Users/ethankuch/my_project/yolov5/runs/detect --test_path /Users/ethankuch/my_project/yolov5/data/foodimage

In [ ]:
# Detect video, change source and data to your directory
!python detect.py --data /Users/ethankuch/my_project/yolov5/models/dataset.yaml --source data/foodvideo/foodvid2.mp4 --weights yolov5s6.pt --img 1280 --conf 0.4 
# display.Video(filename='runs/detect/exp3/foodvid.mp4', width=600)

In [25]:
# !python val.py --data /Users/ethankuch/my_project/yolov5/models/dataset.yaml --weights yolov5s.pt --img 320

val: data=/Users/ethankuch/my_project/yolov5/models/dataset.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=320, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-164-g177da7f torch 1.11.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Traceback (most recent call last):
  File "val.py", line 396, in <module>
    main(opt)
  File "val.py", line 369, in main
    run(**vars(opt))
  File "/Users/ethankuch/opt/anaconda3/envs/visvenv/lib/python3.7/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "val.py", line 167, in run
    assert ncm == nc, f'{weights[0]} ({ncm} classes) trained on different --data than what you passed ({nc} ' \
AssertionError: yolov5s.pt (80 classes) trained on d